# CNN Ensemble - Vote

流れ：

① 複数の条件で学習モデルの weights を保存する。　② 学習したのと同じ空のモデルを複数作成する。　③ 複数の空のモデルに学習済みの weights を入力する。　④ それぞれのモデルで予測を行う。　⑤ 予想結果の多数決を取り、１つの予想結果にする。

### Library の読み込み

In [1]:
import numpy as np # linear algebra
import pandas as pd # 今回は使用する
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [2]:
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler

Using TensorFlow backend.


### ファイル関連定数

In [3]:
test_file = "./data/test.csv"
output_file = "cnn_ensemble.csv"

### モデルの作成

In [4]:
def model_build():
    model = Sequential()

    model.add(Conv2D(filters = 16, kernel_size = (3, 3), activation='relu',
                 input_shape = (28, 28, 1)))
    model.add(BatchNormalization())
    model.add(Conv2D(filters = 16, kernel_size = (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters = 16, kernel_size = (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(strides=(2,2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(filters = 32, kernel_size = (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters = 32, kernel_size = (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters = 32, kernel_size = (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(strides=(2,2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer = Adam(lr=1e-4), metrics=["accuracy"])
    
    return model

### 空のモデルのリストを作成

In [5]:
model_ensemble = []
n = 3 # weights の数（奇数にする）

for i in range(n):
    model_ensemble.append(model_build())

In [6]:
model_ensemble

### 学習済み weights を各モデルに入力

In [7]:
fpath = './ensemble/%d.hdf5'%(i)

for i in range(n):
    model_ensemble[i].model.load_weights(fpath)

D:\Users\301518\AppData\Local\Continuum\anaconda3\envs\Python3.6-gpu-PyTorch\lib\site-packages\keras\engine\sequential.py:110: UserWarning: `Sequential.model` is deprecated. `Sequential` is a subclass of `Model`, you can just use your `Sequential` instance directly.
  warnings.warn('`Sequential.model` is deprecated. '


### テストデータの読み込み

In [8]:
mnist_testset = np.loadtxt(test_file, skiprows=1, dtype='int', delimiter=',')
x_test = mnist_testset.astype("float32")
x_test = x_test.reshape(-1, 28, 28, 1)/255.

### 学習したモデルを用いて予測

In [9]:
y_ensemble = []

for i in range(n):
    y_ensemble.append(np.argmax(model_ensemble[i].predict(x_test, batch_size=64), axis=1))
    print(y_ensemble[i])

[2 0 9 ... 3 9 2]
[2 0 9 ... 3 9 2]
[2 0 9 ... 3 9 2]


In [10]:
y_ensemble = pd.DataFrame(y_ensemble).transpose()
y_ensemble

,0,1,2
0,2,2,2
1,0,0,0
2,9,9,9
3,0,0,0
4,3,3,3
...,...,...,...
27995,9,9,9
27996,7,7,7
27997,3,3,3
27998,9,9,9


### ３つの予測から多数決（再頻度）

In [11]:
y_pred = y_ensemble.mode(axis=1) # 再頻度の抽出
y_pred

,0
0,2
1,0
2,9
3,0
4,3
...,...
27995,9
27996,7
27997,3
27998,9


In [12]:
y_pred = y_pred[0] # DF をリストに変換する単純な方法
y_pred

0        2
1        0
2        9
3        0
4        3
        ..
27995    9
27996    7
27997    3
27998    9
27999    2
Name: 0, Length: 28000, dtype: int64

### 提出用CSVファイルの作成

In [13]:
with open(output_file, 'w') as f :
    f.write('ImageId,Label\n')
    for i in range(len(y_pred)) :
        f.write("".join([str(i+1),',',str(y_pred[i]),'\n']))